In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import numpy.linalg as la

import matplotlib.pyplot as plt
%matplotlib inline

# Trip distribution

The purpose of this notebook is to learn how to work with simple gravity models for trip distribution.

First, let's load the population, commuting, and municipality data from the first exercise:

In [ ]:
df_employment = pd.read_parquet("data/employment.parquet")
df_commutes = pd.read_parquet("data/commutes.parquet")
df_municipalities = gpd.read_parquet("data/municipalities.parquet")

**Task**: As before, reduce all data sets to the area of Île-de-France.

In [ ]:
idf_departments = ["75", "92", "93", "94", "95", "77", "91", "78"]

In [ ]:
# Insert your code here

### SOLUTION START
df_employment = df_employment[df_employment["municipality_id"].str[:2].isin(idf_departments)]
df_municipalities = df_municipalities[df_municipalities["municipality_id"].str[:2].isin(idf_departments)]

df_commutes = df_commutes[
    df_commutes["origin_id"].str[:2].isin(idf_departments) &
    df_commutes["destination_id"].str[:2].isin(idf_departments)
]

**Task**: Keeping track of the order of the data will be important. Set up a fixed list of municipalities and adjust the indices of all data sets. Especially, take care of the commuting data set.

Hint: Make use of `pd.MultiIndex.from_product`

In [ ]:
# Insert your code here

# municipalities = 

# df_emloyment = 
# df_commutes = 

### SOLUTION START
municipalities = df_municipalities["municipality_id"].unique()

df_employment = df_employment.set_index("municipality_id").reindex(municipalities)

df_commutes = df_commutes.set_index(["origin_id", "destination_id"]).reindex(
    pd.MultiIndex.from_product([municipalities, municipalities], names = ["origin_id", "destination_id"]))

Have a look at your data sets after reindexing, do you notice anything special?

**Task**: How many flow values can we theoretically have (between all zones) in Île-de-France? For how many do we have actual values?

In [ ]:
# Insert your code here

### SOLUTION START
len(df_commutes), np.count_nonzero(~df_commutes.isna()), np.count_nonzero(~df_commutes.isna()) / len(df_commutes)

**Task**: Replace missing values by zero (zero commuters).

In [ ]:
# Insert your code here

### SOLUTION START
df_commutes["weight"] = df_commutes["weight"].fillna(0.0)

## Friction term

The first step in setting up our model is to obtain the friction term.

**Task**: The gravity model puts into relation different places in the study area. The friction term describes how easy it is to reach one municipality from another one. The first step is, therefore, to obtain the distances between all zones. Complete the following code to set up a distance matrix `distance_matrix`.

In [ ]:
centroids = df_municipalities["geometry"].centroid
centroids = np.array([centroids.x, centroids.y] ).T

distance_matrix = np.zeros((len(municipalities), len(municipalities)))

for k in range(len(municipalities)):
    # Insert your code here
    # distance_matrix[k,:] = # Calculate the Euclidean distance, you may also try numpy.linalg.norm
    
    ### SOLUTION START
    distance_matrix[k,:] = la.norm(centroids[k] - centroids, axis = 1)

Plot the distance matrix (it may take a while):

In [ ]:
plt.pcolor(distance_matrix)

What do you notice in the plot? Can you explain the structure?

**Task**: Analogously to the distance matrix, we need a flow matrix indicating all observed movements (`weight`) between all zones. Obtain this matrix by transforming the commuting data set into a matrix.

Hint: Have a look at `numpy.ndarray.reshape`

In [ ]:
# Insert your code here
# flow_matrix = 

### SOLUTION START
flow_matrix = df_commutes["weight"].values.reshape((len(municipalities), len(municipalities)))

**Task**: Now we obtain the data to set up the friction model:
- Bin the distances into about twenty distance bins and sum up the commuters you find in each distance bin
- Plot how much flow occurs at every distance bin

In [ ]:
df_friction = pd.DataFrame({
    "distance": distance_matrix.flatten(),
    "flow": flow_matrix.flatten()
})

distance_classes = np.arange(20) * 5000

# Hint: Check numpy.digitize

# Insert your code here

### SOLUTION START
df_friction["distance_class"] = np.digitize(df_friction["distance"], distance_classes)
df_friction = df_friction.groupby("distance_class")["flow"].sum().reset_index()

plt.plot(distance_classes * 1e-3, df_friction["flow"].values)

**Task**: Now divide the obtained flow in each bin by the total flow, to obtain an empirical probability density function (pdf). Plot the function in absolute coordinates and with the probability logarithmized. What do you observe?

In [ ]:
# Insert your code here

# pdf = 

### SOLUTION START
pdf = df_friction["flow"].values / df_friction["flow"].sum()

plt.figure()
plt.plot(distance_classes * 1e-3, pdf)

plt.figure()
plt.plot(distance_classes * 1e-3, np.log(pdf))

**Task**: In logarithmic space, manually (or automatically, if you like), fit a linear function on the graph that you see.

In [ ]:
# Insert code here

# a = ?
# b = ?

# logy = a + b * np.log(pdf)

### SOLUTION START
a = -0.9
b = -1.02e-4

logy = a + b * distance_classes

plt.plot(distance_classes * 1e-3, np.log(pdf))
plt.plot(distance_classes * 1e-3, logy)

**Task**: Now plot the initial data along with your fitted curve in linear space. How does you friction model look like?

In [ ]:
# Insert code here

### SOLUTION START
a = -0.9
b = -1.02e-4

logy = a + b * distance_classes

plt.plot(distance_classes * 1e-3, pdf)
plt.plot(distance_classes * 1e-3, np.exp(logy))

**Task**: Based on your distance matrix and your friction model, calculate a friction matrix:

In [ ]:
# Insert code here

# friction_matrix =

### SOLUTION START 
friction_matrix = np.exp(a + b * distance_matrix)

## Double-constrained gravity model

Let's move on to the double-constrained model. In that model, both the origin and destination flows $O_i$ and $D_j$ are known and we aim to automatically find the attraction and production terms $A_j$ and $P_i$.

The model is defined as follows:

$$
F_{ij} = \frac{O_i \cdot D_i}{(\sum_i P_i \cdot \rho_{ij})\cdot (\sum_j A_j \cdot \rho_{ij})}
$$

the attraction and production terms are obtained by iteratively executing:

$$
P_i = \frac{O_i}{\sum_j A_j \cdot \rho_{ij}}
$$
$$
A_j = \frac{D_j}{\sum_i P_i \cdot \rho_{ij}}
$$

**Task**: Implement the double-constrained gravity model to calculate the production and attraction terms. Optionally, you may plot the delta of the production and attraction terms in every iteration to observe convergence. The model should be well converged after 25 iterations.

In [ ]:
### Insert code here

# origins = # Format properly
# destinations = # Format properly

# production = # Initialize to one
# attraction = # Initialize to one

# for iteration in range(500):
#    for i in range(len(municipalities)):
        # ...

#    for j in range(len(municipalities)):
        # ...

### SOLUTION START 
origins = np.sum(flow_matrix, axis = 1)
destinations = np.sum(flow_matrix, axis = 0)

production = np.ones((len(municipalities),))
attraction = np.ones((len(municipalities),))

delta_production = []
delta_attraction = []

for iteration in range(25):
    delta_production_iteration = []
    delta_attraction_iteration = []

    for i in range(len(municipalities)):
        update = origins[i] / np.sum(attraction * friction_matrix[i,:])
        delta_production_iteration.append(update - production[i])
        production[i] = update
    
    for j in range(len(municipalities)):
        update = destinations[j] / np.sum(production * friction_matrix[:,j])
        delta_attraction_iteration.append(update - attraction[j])
        attraction[j] = update   

    delta_production.append(np.max(np.abs(delta_production_iteration)))
    delta_attraction.append(np.max(np.abs(delta_attraction_iteration)))

plt.figure()
plt.plot(delta_attraction)

plt.figure()
plt.plot(delta_production)

**Task**: Calculate the resulting flows of your model. Compare the flows with the reference data, and also compare origin and destination flows in two additional plots.

In [ ]:
### Insert code here

### SOLUTION START 
F = np.copy(friction_matrix)

for i in range(len(municipalities)):
    F[i,:] *= origins[i] / np.sum(attraction * friction_matrix[i,:])

for j in range(len(municipalities)):
    F[:,j] *= destinations[j] / np.sum(production * friction_matrix[:,j])

plt.figure()
plt.plot(flow_matrix.flatten(), F.flatten(), ".")
plt.plot([0, np.max(flow_matrix)], [0, np.max(flow_matrix)], "k--")

plt.figure()
plt.plot(np.sum(flow_matrix, axis = 1), np.sum(F, axis = 1), ".")
plt.plot([0, np.max(np.sum(flow_matrix, axis = 1))], [0, np.max(np.sum(flow_matrix, axis = 1))], "k--")

plt.figure()
plt.plot(np.sum(flow_matrix, axis = 0), np.sum(F, axis = 0), ".")
plt.plot([0, np.max(np.sum(flow_matrix, axis = 0))], [0, np.max(np.sum(flow_matrix, axis = 0))], "k--")

What do you observe? Which municipalities could be the outliers on the bottom of the flow comparison?

**Task**: Try to estimate a new model using the following modified friction term:

$$
F'_{ij} = \begin{cases}
    F_{ij} & \text{if} \ i \neq j \\
    F_{ij} + \gamma & \text{if} \  i = j
\end{cases}
$$

Which parameter $\gamma$ works best?

Hint: Keep your existing friction matrix in `friction_matrix` and create new matrices on the fly for testing.

In [ ]:
### Insert code here

### SOLUTION START
gammas = np.linspace(0, 10, 5)

for gamma in gammas:
    updated_friction_matrix = friction_matrix + np.eye(len(municipalities)) * gamma

    origins = np.sum(flow_matrix, axis = 1)
    destinations = np.sum(flow_matrix, axis = 0)
    
    production = np.ones((len(municipalities),))
    attraction = np.ones((len(municipalities),))
    
    for iteration in range(25):
        for i in range(len(municipalities)):
            production[i] = origins[i] / np.sum(attraction * updated_friction_matrix[i,:])
        
        for j in range(len(municipalities)):
            attraction[j] = destinations[j] / np.sum(production * updated_friction_matrix[:,j])

    F = np.copy(updated_friction_matrix)

    for i in range(len(municipalities)):
        F[i,:] *= origins[i] / np.sum(attraction * updated_friction_matrix[i,:])
    
    for j in range(len(municipalities)):
        F[:,j] *= destinations[j] / np.sum(production * updated_friction_matrix[:,j])
    
    plt.figure()
    plt.title("gamma={}".format(gamma))
    plt.plot(flow_matrix.flatten(), F.flatten(), ".")
    plt.plot([0, np.max(flow_matrix)], [0, np.max(flow_matrix)], "k--")

**Task**: Do you remeber the initial data frame `df_commutes`? Add a new column to this data frame into which you write your latest modeling results. Show the data frame. Calculate how many zero-cells have been filled with a non-zero value.

In [ ]:
### Insert code here

### SOLUTION START
df_flow = df_commutes.copy()
df_flow["model"] = F.reshape((-1,))
df_flow

np.mean((df_flow["weight"] == 0.0) & (df_flow["model"] != 0.0))

**Task**: On map, show the commuting outflow from Melun like in the previous notebook, but using the model data. For better visibility, put the maximum value of the color scale to 1000 or 500.

In [ ]:
# Insert your code here

### SOLUTION START
df = df_flow.copy().reset_index()
df = df[df["origin_id"] == "77288"]
df = df.groupby("destination_id")["weight"].sum().reset_index()
df = df.rename(columns = { "destination_id": "municipality_id" })

pd.merge(df_municipalities[df_municipalities["municipality_id"].str[:2].isin([
    "75", "92", "93", "94", "95", "77", "91", "78"
])], df).plot("weight", legend = True, vmax = 500)

Save the data set for the next exercise.

In [ ]:
df_flow.to_parquet("data/flow.parquet")

**Congratulations!** You can now solve exercise 2.3 of the course project.